# Example: Class with `Streamer` method overridden

## Standalone object

We define a class `TOverrideStreamer` with 2 member variables:

```c++
class TOverrideStreamer : public TObject {
    int m_int{ 0 };
    double m_double{ 0.0 };

    // ... other codes ...
};
```

Its `Streamer` method is overridden, with additionally writting a "tag" `0x12345678`:

```c++
void TOverrideStreamer::Streamer( TBuffer& b ) {
    if ( b.IsWriting() )
    {
        TObject::Streamer( b ); // Call base class Streamer
        b << m_int;
        b << 0x12345678; // Write a tag
        b << m_double;
    }
    else
    {
        // ... reading logic ...
    }
}
```

### `Uproot` cannot handle it natively

When reading a `TOverrideStreamer` object with `Uproot`, it will raise an error because `Uproot` does not know how to handle the overridden `Streamer` method.

In [ ]:
import uproot

f = uproot.open("demo-data.root")
b = f["my_tree/standalone_obj"]
b.show(name_width=40, typename_width=40)

In [ ]:
try:
    b.array()
except Exception as e:
    print(repr(e))

## How `uproot-custom` reads it

```mermaid
flowchart TD
    subgraph py["Python field"]
        direction TB
        AsCustom --> fac["Factory (Primitive, STLVector, TString, ...)"]
        fac["Factory (Primitive, STLVector, TString, ...)"] -- Optional --> form(["construct awkward forms"])
        fac --> build_reader(["build corresponding C++ reader"])
        fac --> build_ak(["construct awkward arrays"])
    end

    user_fac["User's Factory"] -. Register .-> fac

    subgraph cpp["C++ field"]
        direction TB
        build_reader --> reader["C++ Reader"]
        reader --> read_bin(["read binary data"])
        read_bin --> ret_data(["return data"])
    end

    ret_data --> raw_data[("tuple, list, numpy arrays, ...")]
    raw_data --> build_ak
```

### C++ "reader" for `TOverrideStreamer`

To read the `TOverrideStreamer` object correctly, we need to implement a custom reader in `uproot-custom` that knows how to skip the tag during reading:

```c++
class OverrideStreamerReader : public IElementReader {
  private:
    std::shared_ptr<std::vector<int>> m_data_ints;
    std::shared_ptr<std::vector<double>> m_data_doubles;
    // ... other codes ...

  public:
    void read( BinaryBuffer& buffer ) {
        buffer.skip_TObject();

        m_data_ints->push_back( buffer.read<int>() );

        auto tag = buffer.read<uint32_t>();
        if ( tag != 0x12345678 ) throw std::runtime_error( "Error: Unexpected tag value." );

        m_data_doubles->push_back( buffer.read<double>() );
    }

    py::object data() const {
        auto int_array    = make_array( m_data_ints );
        auto double_array = make_array( m_data_doubles );
        return py::make_tuple( int_array, double_array );
    }

    // ... other codes ...
};
```


### Python "factory" for `TOverrideStreamer`

Python factory receives the raw data from the C++ reader and constructs the corresponding `awkward` array and form:

```python
class OverrideStreamerFactory(Factory):
    # ... other codes ...

    def make_awkward_content(self, raw_data):
        int_array, double_array = raw_data

        return awkward.contents.RecordArray(
            [
                awkward.contents.NumpyArray(int_array),
                awkward.contents.NumpyArray(double_array),
            ],
            ["m_int", "m_double"],
        )
```

In [ ]:
import uproot_custom

from my_reader import OverrideStreamerFactory

# Register custom factory
uproot_custom.registered_factories.add(OverrideStreamerFactory)

# Specify target branches
uproot_custom.AsCustom.target_branches.add("/my_tree:standalone_obj")

In [ ]:
f = uproot.open("demo-data.root")
b = f["my_tree/standalone_obj"]
b.show(name_width=40, typename_width=40)

In [ ]:
b.array()

## Reusing the custom reader and factory

Once the custom reader and factory are implemented, `uproot-custom` can automatically use them to read more complex data structures that contain `TOverrideStreamer` objects, such as:

- `std::vector<TOverrideStreamer>`
- `std::map<int, TOverrideStreamer>`
- ...

In [ ]:
uproot_custom.AsCustom.target_branches |= {
    "/my_tree:my_obj/m_objVec",
    "/my_tree:my_obj/m_objMap/m_objMap.second",
}

In [ ]:
f = uproot.open("demo-data.root")
b = f["my_tree/my_obj"]
b.show(name_width=40, typename_width=40, filter_name="m_obj*")

In [ ]:
b.arrays(filter_name="m_obj*")

## Using `uproot.dask`

Users know concrete structure of `awkward` contents, so the factory can also build the `awkward` form:

```python
class OverrideStreamerFactory(Factory):
    # ... other codes ...
    def make_awkward_form(self):
        return awkward.forms.RecordForm(
            [
                awkward.forms.NumpyForm("int32"),
                awkward.forms.NumpyForm("float64"),
            ],
            ["m_int", "m_double"],
        )
```

so we can even use `uproot.dask` to read:

In [ ]:
arr_dask = uproot.dask("demo-data.root:my_tree/standalone_obj")
arr_dask.type

In [ ]:
arr_dask.compute()